Used ChatGPT to find different nautical python libraries and help with pandas syntax.

Used this Medium article https://medium.com/shipping-intel/maritime-professionals-top-5-python-libraries-4e1189ca4207 to find more python libraries

Import needed libraries

In [1]:
import pandas as pd
from geographiclib.geodesic import Geodesic
import math
import folium
geod = Geodesic.WGS84 

Import dataframe of data. The data must be a csv and have a lat, long, COG, SOC, MMSI, and date

In [2]:
DATA = 'Dummy Dataset.csv'
orig_df = pd.read_csv(DATA)
orig_df.head()

,MMSI,BaseDateTime,LAT,LON,SOG,COG
0,338075892,2024-01-01T00:00:00,42.36,-70.670000,5,90
1,338075892,2024-01-01T00:00:01,42.36,-70.668120,5,90
2,338075892,2024-01-01T00:00:02,42.36,-70.666241,5,90
3,338075892,2024-01-01T00:00:03,42.36,-70.664361,5,90
4,338075892,2024-01-01T00:00:04,42.36,-70.662482,5,90


converts kts (nm/hr) to meters/sec

In [3]:
def speed_conversion(speed_in_kts):
    return (0.514444 * speed_in_kts)

Creates a smaller dataframe only including data that we need to calculate dead reckoning, cleans dataframe. Sorts by vessel and time.

In [4]:
df = orig_df[['MMSI', 'BaseDateTime', 'LAT', 'LON', 'SOG', 'COG']]
df['BaseDateTime'] = pd.to_datetime(df['BaseDateTime'])
df['SOG_convert'] = orig_df['SOG'].apply(speed_conversion)
df = df.sort_values(['MMSI', 'BaseDateTime'])

Creates actual lat/long to compare predictions to

In [5]:
df[['next_BaseDateTime', 'future_lat_actual', 'future_lon_actual']] = (df.groupby('MMSI')[['BaseDateTime', 'LAT', 'LON']].shift(-1))
df = df.dropna(subset=["next_BaseDateTime"])
df['time_change'] = (df['next_BaseDateTime'] - df['BaseDateTime']).dt.total_seconds()
df['distance'] = df['SOG_convert'] * df['time_change']

Function Predicts Lat/Long in 1 second

In [6]:
def calc_future_position(lat, lon, cog, dist_in_sec):
    g = geod.Direct(lat, lon, cog, dist_in_sec)
    return (g['lat2'], g['lon2'])

In [7]:
df['future_pred'] = df.apply(lambda x: calc_future_position(x['LAT'], x['LON'], x['COG'], x['distance']), axis=1)
df[['future_lat_pred', 'future_lon_pred']] = df['future_pred'].apply(pd.Series)
df['mse_calc_lat'] = (df['future_lat_actual'] - df['future_lat_pred']) ** 2
df['mse_calc_lon'] = (df['future_lon_actual'] - df['future_lon_pred']) ** 2
n = df.shape[0]

mse_lat = df['mse_calc_lat'].sum() / n
mse_lon = df['mse_calc_lon'].sum() / n

In [8]:
print(mse_lat)
print(mse_lon)

1.8114821225869072e-23
3.4163333708622418e-06


In [9]:
df.head()

,MMSI,BaseDateTime,LAT,LON,SOG,COG,SOG_convert,next_BaseDateTime,future_lat_actual,future_lon_actual,time_change,distance,future_pred,future_lat_pred,future_lon_pred,mse_calc_lat,mse_calc_lon
0,338075892,2024-01-01 00:00:00,42.36,-70.670000,5,90,2.57222,2024-01-01 00:00:01,42.36,-70.668120,1.0,2.57222,"(42.35999999999574, -70.66996877696971)",42.36,-70.669969,1.811482e-23,0.000003
1,338075892,2024-01-01 00:00:01,42.36,-70.668120,5,90,2.57222,2024-01-01 00:00:02,42.36,-70.666241,1.0,2.57222,"(42.35999999999574, -70.66808877696971)",42.36,-70.668089,1.811482e-23,0.000003
2,338075892,2024-01-01 00:00:02,42.36,-70.666241,5,90,2.57222,2024-01-01 00:00:03,42.36,-70.664361,1.0,2.57222,"(42.35999999999574, -70.66620977696971)",42.36,-70.666210,1.811482e-23,0.000003
3,338075892,2024-01-01 00:00:03,42.36,-70.664361,5,90,2.57222,2024-01-01 00:00:04,42.36,-70.662482,1.0,2.57222,"(42.35999999999574, -70.66432977696971)",42.36,-70.664330,1.811482e-23,0.000003
4,338075892,2024-01-01 00:00:04,42.36,-70.662482,5,90,2.57222,2024-01-01 00:00:05,42.36,-70.660602,1.0,2.57222,"(42.35999999999574, -70.6624507769697)",42.36,-70.662451,1.811482e-23,0.000003


In [12]:
def add_marker(lat, lon, my_color):
    folium.CircleMarker(location=[lat, lon], radius=1, color= my_color).add_to(map)

Code taken from Medium Article

In [19]:
# Save coordinates to Feather
pred_data = df[['future_lat_pred', 'future_lon_pred']]
pred_data.to_feather('coordinates.feather')

# Load coordinates from Feather
loaded_df = pd.read_feather('coordinates.feather')

# Create a map centered around the coordinates
map = folium.Map(location=[loaded_df['future_lat_pred'][0], loaded_df['future_lon_pred'][0]], zoom_start=12)

# Add a marker for the location
loaded_df.apply(lambda x: add_marker(x['future_lat_pred'], x['future_lon_pred'], 'red'), axis=1)

actual_data = df[['future_lat_actual', 'future_lon_actual']]
actual_data.to_feather('coordinates.feather')

# Load coordinates from Feather
loaded_df = pd.read_feather('coordinates.feather')

# Add a marker for the location
loaded_df.apply(lambda x: add_marker(x['future_lat_actual'], x['future_lon_actual'], 'black'), axis=1)

map

Save Map (optional)

In [ ]:
# Display the map
map.save('map.html')